<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/UTXO_data_analysis_Task_3_with_extended_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime

## Import Data from Google Drive and Data Wrangling

In [ ]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

# Mounting drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
df_2010=pd.read_csv('/content/drive/My Drive/joint_2010.csv')
df_2010.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
df_2011=pd.read_csv('/content/drive/My Drive/joint_2011.csv')
df_2011.tail()

,num,value,block_date,spent_block_date
1048570,415429,8.496683e+06,8/6/2011,8/8/2011
1048571,415965,1.200000e+08,8/6/2011,8/8/2011
1048572,416087,1.700000e+07,8/6/2011,8/8/2011
1048573,416508,5.884410e+07,8/6/2011,8/8/2011
1048574,420271,1.222000e+09,8/6/2011,8/8/2011


In [ ]:
merge = [df_2010, df_2011]
df = pd.concat(merge)
df.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
# Generate the UTXO value in bitcoin unit, which = $value/10^{8}$
df['UTXO'] = df['value']*10**(-8)
df = df.reset_index()
df = df.drop(['value','index','num'], axis = 1)
df.tail()

,block_date,spent_block_date,UTXO
1186094,8/6/2011,8/8/2011,0.084967
1186095,8/6/2011,8/8/2011,1.200000
1186096,8/6/2011,8/8/2011,0.170000
1186097,8/6/2011,8/8/2011,0.588441
1186098,8/6/2011,8/8/2011,12.220000


In [ ]:
# Change the block_date and spent_block_date to datatime object
df['block_date'] = pd.to_datetime(df['block_date'], format='%m/%d/%Y')
df['spent_block_date'] = pd.to_datetime(df['spent_block_date'], format='%m/%d/%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186099 entries, 0 to 1186098
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   block_date        1186099 non-null  datetime64[ns]
 1   spent_block_date  1164547 non-null  datetime64[ns]
 2   UTXO              1186099 non-null  float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 27.1 MB


#Task 3: Calculate the distribution of Life Expectancy
Sorted by day, month, as well as the year, and print a plot by every time slot.


In [ ]:
from datetime import datetime
df['Life_Expectancy'] = df['spent_block_date'] - df['block_date']
df['Life_Expectancy'] = df['Life_Expectancy'].map(lambda x:x.days)
df['WALE']=(df['UTXO'].mul(df['Life_Expectancy'])).cumsum().div(df['UTXO'].cumsum())
df.tail()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE
1164542,2011-08-06,2011-08-07,137.097721,1,110.014642
1164543,2011-08-06,2011-08-07,0.029247,1,110.014642
1164544,2011-08-06,2011-08-07,0.530000,1,110.014640
1164545,2011-08-06,2011-08-07,0.091483,1,110.014640
1164546,2011-08-06,2011-08-08,12.220000,2,110.014603


In [ ]:
df.reset_index(drop = True, inplace = True)
df.insert(5,'< 1d','')
df.insert(6,'1d ~ 1m','')
df.insert(7,'1m ~ 1q','')
df.insert(8,'1q ~ 6m','')
df.insert(9,'6m ~ 1y','')
df.insert(10,'1y ~ 2y','')
df.insert(11,'2y ~ 3y','')
df.insert(12,'3y ~ 4y','')
df.insert(13,'4y ~ 5y','')
df.insert(14, '> 5y', '')
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,2009-01-12,50.0,3,3.000000,,,,,,,,,,
1,2009-01-11,2011-06-14,50.0,884,443.500000,,,,,,,,,,
2,2009-01-12,2010-11-26,50.0,683,523.333333,,,,,,,,,,
3,2009-01-12,2010-11-16,10.0,673,532.687500,,,,,,,,,,
4,2009-01-12,2009-01-14,1.0,2,529.391304,,,,,,,,,,


In [ ]:
numrows = df.shape[0]
for i in range(numrows):
  if df.loc[i,'Life_Expectancy'] < 1:
    df.loc[i, '< 1d'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 30:
    df.loc[i,'1d ~ 1m'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 91:
    df.loc[i,'1m ~ 1q']= df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 182:
    df.loc[i, '1q ~ 6m'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 365:
    df.loc[i, '6m ~ 1y'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 365*2:
    df.loc[i,'1y ~ 2y'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 365*3:
    df.loc[i, '2y ~ 3y'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 365*4:
    df.loc[i, '3y ~ 4y'] = df.loc[i,'UTXO']
  elif df.loc[i,'Life_Expectancy'] < 365*5:
    df.loc[i, '4y ~ 5y'] = df.loc[i,'UTXO']
  else:
    df.loc[i,'> 5y'] = df.loc[i,'UTXO']
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,2009-01-12,50.0,3,3.000000,,50,,,,,,,,
1,2009-01-11,2011-06-14,50.0,884,443.500000,,,,,,,50,,,
2,2009-01-12,2010-11-26,50.0,683,523.333333,,,,,,50,,,,
3,2009-01-12,2010-11-16,10.0,673,532.687500,,,,,,10,,,,
4,2009-01-12,2009-01-14,1.0,2,529.391304,,1,,,,,,,,


In [ ]:
### we need to sort the data by spent block data since they only count into the distribution after they are spent
df = df.sort_values(by ='spent_block_date')
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,2009-01-12,50.0,3.0,3.000000,,50,,,,,,,,
11,2009-01-12,2009-01-12,40.0,0.0,1.250000,40,,,,,,,,,
10,2009-01-12,2009-01-12,29.0,0.0,1.898734,29,,,,,,,,,
7,2009-01-12,2009-01-12,28.0,0.0,1.013514,28,,,,,,,,,
6,2009-01-12,2009-01-12,30.0,0.0,0.842697,30,,,,,,,,,


In [ ]:
### for each UTXO, we get the accumulated sum of spent UTXO that satisfy this range of life expectency
df.columns

Index(['block_date', 'spent_block_date', 'UTXO', 'Life_Expectancy', 'WALE',
       '< 1d', '1d ~ 1m', '1m ~ 1q', '1q ~ 6m', '6m ~ 1y', '1y ~ 2y',
       '2y ~ 3y', '3y ~ 4y', '4y ~ 5y', '> 5y'],
      dtype='object')

In [ ]:
cols = ['< 1d', '1d ~ 1m', '1m ~ 1q', '1q ~ 6m', '6m ~ 1y', '1y ~ 2y','2y ~ 3y', '3y ~ 4y', '4y ~ 5y', '> 5y']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df=df.fillna(0)
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,2009-01-12,50.0,3.0,3.000000,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2009-01-12,2009-01-12,40.0,0.0,1.250000,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2009-01-12,2009-01-12,29.0,0.0,1.898734,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2009-01-12,2009-01-12,28.0,0.0,1.013514,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2009-01-12,2009-01-12,30.0,0.0,0.842697,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df[cols]=df[cols].cumsum()
df.head()

,block_date,spent_block_date,UTXO,Life_Expectancy,WALE,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,2009-01-12,50.0,3.0,3.000000,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2009-01-12,2009-01-12,40.0,0.0,1.250000,40.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2009-01-12,2009-01-12,29.0,0.0,1.898734,69.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2009-01-12,2009-01-12,28.0,0.0,1.013514,97.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2009-01-12,2009-01-12,30.0,0.0,0.842697,127.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
### for each date, we count until the last UTXO spent
df_DLE=df.groupby('spent_block_date')[cols].max().reset_index()
df_DLE.head()

,spent_block_date,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-12,129.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009-01-14,129.0,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009-01-15,229.0,511.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009-01-16,229.0,711.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009-01-18,229.0,861.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
### we need to fill the missing dates
df_UTXO.head()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum,WALE
3,2009-01-12,4879.0,179.0,4700.0,13100.0,3.000000
4,2009-01-13,6150.0,0.0,6150.0,19250.0,3.000000
5,2009-01-14,6511.0,61.0,6450.0,25700.0,0.925000
6,2009-01-15,6800.0,500.0,6300.0,32000.0,1.338462
7,2009-01-16,5600.0,200.0,5400.0,37400.0,1.353191


In [ ]:
df_DLE=df_UTXO.merge(df_DLE, how='left', left_on='date',right_on='spent_block_date')
df_DLE=df_DLE.fillna(method='ffill')
df_DLE.head()

,date,UTXO_newborn,UTXO_dead,Net_New,UTXO_Cum,WALE,spent_block_date,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-12,4879.0,179.0,4700.0,13100.0,3.000000,2009-01-12,129.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009-01-13,6150.0,0.0,6150.0,19250.0,3.000000,2009-01-12,129.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2009-01-14,6511.0,61.0,6450.0,25700.0,0.925000,2009-01-14,129.0,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2009-01-15,6800.0,500.0,6300.0,32000.0,1.338462,2009-01-15,229.0,511.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009-01-16,5600.0,200.0,5400.0,37400.0,1.353191,2009-01-16,229.0,711.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import plotly.offline as py     
import plotly.graph_objects as go

trace0 = go.Scatter(x = df_DLE["date"], y = df_DLE["< 1d"], name = '< 1d')
trace1 = go.Scatter(x = df_DLE["date"], y = df_DLE["1d ~ 1m"], name = '1d ~ 1m')
trace2 = go.Scatter(x = df_DLE["date"], y = df_DLE["1m ~ 1q"], name = '1m ~ 1q')
trace3 = go.Scatter(x = df_DLE["date"], y = df_DLE["1q ~ 6m"], name = '1q ~ 6m')
trace4 = go.Scatter(x = df_DLE["date"], y = df_DLE["6m ~ 1y"], name = '6m ~ 1y')
trace5 = go.Scatter(x = df_DLE["date"], y = df_DLE["1y ~ 2y"], name = '1y ~ 2y')
trace6 = go.Scatter(x = df_DLE["date"], y = df_DLE["2y ~ 3y"], name = '2y ~ 3y')
trace7 = go.Scatter(x = df_DLE["date"], y = df_DLE["3y ~ 4y"], name = '3y ~ 4y')
trace8 = go.Scatter(x = df_DLE["date"], y = df_DLE["4y ~ 5y"], name = '4y ~ 5y')
trace9 = go.Scatter(x = df_DLE["date"], y = df_DLE["> 5y"], name = '> 5y')

layout = go.Layout(title="life Expectancy Distribution", xaxis = dict(title="date"), yaxis = dict(title ="Life Expectancy Distribution") ) 
data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]

fig = go.Figure(layout = layout, data = data) 
fig.update_yaxes(type="log")



# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

py.iplot(fig)